In [1]:
# f_seed 需改 PY/init 中的 因为 models_logd 传入的是 global 的f_seed 值
from PYmodule import *
from PYmodule.models_logd import *

In [2]:
f_seedlabel = 'f%d'%abs(int(np.log10(f_seed)))
f_seedlegend = r'$\mathrm{f_{seed}}=$'+str(f_seed)
fname = '../M0r8_'+f_seedlabel+'.h5'
prex = '../' + f_seedlabel
prex = './' + f_seedlabel

labels = [r'$\tau$', r'$\log \delta$', r'$\lambda_0$', r'$\alpha$']

reader = emcee.backends.HDFBackend(fname)
ndim = len(labels)

tau = reader.get_autocorr_time(tol=1)
tau = np.max(tau); print('max tau:',tau)
Nburnin = int(3*tau)
Nthin = int(tau/2)

Nburnin = 0
Nthin = 1

samples = reader.get_chain(discard=Nburnin)
probs = reader.get_log_prob(discard=Nburnin)
print('len of samples:', len(samples))

samples = reader.get_chain(discard=Nburnin, thin=Nthin, flat=True)
probs = reader.get_log_prob(discard=Nburnin, thin=Nthin, flat=True)
print('len of extracted samples:', len(samples))
theta_max = samples[np.argmax(probs)]
print('best paras:',theta_max,np.max(probs))


max tau: 249.58268579763228
len of samples: 5000
len of extracted samples: 500000
best paras: [18.7555167  -1.2574505   0.87372563  0.20389703] -3.1054538991409824


In [3]:
# fig = corner.corner(samples,show_titles=True,title_kwargs={'fontsize':15},
# label_kwargs={'fontsize':20},max_n_ticks=4,labels=labels,plot_datapoints=True,
# quantiles=[0.16, 0.5, 0.84])
# axes = np.array(fig.axes).reshape((ndim, ndim))
# for i in range(ndim):
#     for j in range(ndim):
#         axes[i][j].tick_params(labelsize=12)
# plt.savefig(prex+'_corner.png',dpi=400,rasterized=True)
# exit(0)

print(f_seed)

0.01


In [4]:
ndraw = 5

best_model = model(theta_max)

draw = np.floor(np.random.uniform(0,len(samples),size=ndraw)).astype(int)
thetas = samples[draw]
model_thetas = [model(theta_i) for theta_i in thetas]

In [5]:
fig, ax = plt.subplots(figsize=(10, 10))
curve_name = 'MF'
xs = best_model['M_BH'][::int(N_mf/100)] # ~100 points
y_data = best_model[curve_name+'_data'][::int(N_mf/100)]
y_logdata = np.log10(y_data)
y_best = best_model[curve_name][::int(N_mf/100)]
y_err = best_model[curve_name+'_data_err'][::int(N_mf/100)]
ax.plot(xs, y_data, c='C0',label='Willott+ 10')
# error band of W10
ax.fill_between(xs,pow(10.,y_logdata-y_err/2.),pow(10.,y_logdata+y_err/2.),color='C0',alpha=0.1,label='_data error')

mod_list = []
for i in range(ndraw):
    mod = model_thetas[i][curve_name][::int(N_mf/100)]
    mod_list.append(mod)
    # ax.plot(xs, mod, c='grey',label='_',alpha=.2)
spread = np.std(mod_list,axis=0)
med_model = np.median(mod_list,axis=0)
ax.fill_between(xs,med_model-spread,med_model+spread,color='grey',alpha=0.1,label=r'_$1\sigma$ Posterior Spread')
ax.plot(xs, y_best, c='black', label='Best-fit Model')

ax.text(1e9,3e-7, f_seedlegend+'\n'+ \
labels[0]+' = %.2f Myr\n'%(theta_max[0])+labels[1]+' = %.2f\n'%(theta_max[1])\
+labels[2]+' = %.2f\n'%(theta_max[2])+labels[3]+' = %.2f\n'%(theta_max[3])
, fontsize=21)

ax.set_xlim(1e7,1e10); ax.set_xscale('log')
ax.set_ylim(1e-10,1e-4); ax.set_yscale('log')
ax.legend(fontsize=fslegend)
plt.xlabel(r'$\mathrm{M_{\bullet}~(M_\odot)}$',fontsize=fslabel)
plt.ylabel(r'$\mathrm{\Phi_M~(Mpc^{-3}dex^{-1})}$',fontsize=fslabel)
plt.tick_params(labelsize=fstick)

plt.savefig(f_seedlabel+'ndraw%dMF_spread.png'%ndraw,dpi=300,bbox_inches='tight')


# ---------------------------------         QLF         ----------------------------------------


In [6]:
# G15 3 data points 
M1450_G = np.array([-19,-20,-21]); logPhi_G = np.array([-5.2,-5.1,-5.7])
logPhicorr_G = np.array([-4.7,-4.7,-5.7])
print( M1450_G, pow(10,logPhi_G) )

# Giallongo 2019; X-ray faint QLF 4 data points
M1450_G19 = np.array([-19,-20,-21,-22]); Phi_G19 = np.array([7.27,4.77,0.69,0.62])*1e-6
Phip_G19 = np.array([7.12,3.79,1.61,1.44])*1e-6; Phim_G19 = np.array([4.02,2.31,0.60,0.54])*1e-6
# QLF sample: z=5.5 to 6.1; z=5.55 to z=6; extrapolating w/ Jiang2016 density evolution slope: -0.72
Phi_G19 *= pow(10, -0.72*(6-5.55))
Phip_G19*= pow(10, -0.72*(6-5.55))
Phim_G19*= pow(10, -0.72*(6-5.55))

# Jiang 2022;
namelist = [#'fig3a_data_individualFields',
           'fig3a_data_combinedFields_95CL',
           #'fig3a_data_combinedFields_75CL',
           ]
nameFitlist = [#'fig3a_data_modelFit_75CL',
           'fig3a_data_modelFit_95CL',
           ]
name = 'fig3a_data_combinedFields_95CL'
T_J = ascii.read('../data/Jiang_SD_Fig3/%s.txt'%(namelist[0]), guess=False, delimiter=' ',names=['M1450','Phi_cumu'])
M1450_J = (T_J['M1450'][1:] + T_J['M1450'][:-1]) /2.
dM1450_J = T_J['M1450'][1:] - T_J['M1450'][:-1]
Phi_J = 1e-9* (T_J['Phi_cumu'][1:] - T_J['Phi_cumu'][:-1])/dM1450_J

print(M1450_J,'\n', dM1450_J)

[-19 -20 -21] [6.30957344e-06 7.94328235e-06 1.99526231e-06]
       M1450       
-------------------
            -21.125
-20.435000000000002
            -19.705 
       M1450       
------------------
0.7100000000000009
0.6699999999999982
0.7900000000000027


In [8]:
fig, ax = plt.subplots(figsize=(10, 10))
curve_name = 'LF'
xs = best_model['M1450']
x_data = best_model['M1450_data']
y_data = best_model[curve_name+'_data']
y_data_err = best_model[curve_name+'_data_err']
y_best = best_model[curve_name]
# spread
mod_list = []
for i in range(ndraw):
    mod = model_thetas[i][curve_name]
    mod_list.append(mod)
    # ax.plot(xs, mod, c='grey',label='_',alpha=.2)
spread = np.std(mod_list,axis=0)
med_model = np.median(mod_list,axis=0)
ax.fill_between(xs,(med_model-spread)/1e9,(med_model+spread)/1e9,color='grey',alpha=0.1,label='_',zorder=5)
ax.plot(xs, y_best/1e9, c='black', label='_best fit')
ax.plot(xs, y_best/1e9*corr_U14D20(xs),'--', c='black', label='_best fit w/o obscuration correction',zorder=5)

# Matsuoka 2018; data(errorbar) + fitting curve
ax.errorbar(x_data, y_data/1e9, yerr=y_data_err/1e9,fmt='o',capsize=10, label='Matsuoka +18')
ax.plot(xs,LF_M1450(xs),c='C0',label='_Matsu fitting')

# Giallongo 2015; data + Poission errors
# ax.errorbar(M1450_G, pow(10,logPhicorr_G), yerr=pow(10,logPhicorr_G)/np.array([1,3**.5,1]),fmt='s',capsize=10, label='Giallongo+15')
# Giallongo 2019; Phi_corr data + errors
ax.errorbar(M1450_G19, Phi_G19, yerr=[Phim_G19,Phip_G19],fmt='s',capsize=10, label='Giallongo+19')

# Jiang 2022; upper limits
# for name in namelist:
#     T_J = ascii.read('../data/Jiang_SD_Fig3/%s.txt'%name, guess=False, delimiter=' ',names=['M1450','Phi'])
#     M1450_J = T_J['M1450']
#     logPhi_J = T_J['Phi']/1e9
ax.errorbar( M1450_J, Phi_J,xerr=dM1450_J/2, yerr = Phi_J/2,uplims=True, label='Jiang+ 22',fmt='D',color='C2',capsize=3)

# for name in nameFitlist:
#     T_J = ascii.read('../data/Jiang_SD_Fig3/%s.txt'%name, guess=False, delimiter=' ',names=['M1450','Phi'])
#     M1450_J = T_J['M1450']
#     logPhi_J = T_J['Phi']/1e9
#     ax.plot( M1450_J, logPhi_J, c='C2')

ax.set_xlim(-18.5,-30)
# ax.set_ylim(5e-3,1e2)
ax.set_yscale('log')
plt.xlabel(r'$\mathrm{M_{1450}}$',fontsize=fslabel)
plt.ylabel(r'$\mathrm{\Phi~(Mpc^{-3}mag^{-1})}$',fontsize=fslabel)
plt.tick_params(labelsize=fstick)
# plt.tight_layout()
plt.legend(fontsize=fslabel)


locmajx = FixedLocator(np.arange(-17,-30,-2)) # subs=(0.2,0.4,0.6,0.8)
locminx = FixedLocator(np.arange(17,-30,-.5)) # subs=(0.2,0.4,0.6,0.8)
ax.xaxis.set_major_locator(locmajx)
ax.xaxis.set_minor_locator(locminx)
ax.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

locmajy = LogLocator(base=10,numticks=100)
locminy = LogLocator(base=10,subs=np.arange(2, 10) * .1,numticks=100) # subs=(0.2,0.4,0.6,0.8)
ax.yaxis.set_major_locator(locmajy)
ax.yaxis.set_minor_locator(locminy)
ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

plt.savefig(f_seedlabel+'ndraw%dLF_spread.png'%ndraw,dpi=300,bbox_inches='tight')

# prev

In [ ]:
from PYmodule import *
from PYmodule.models_logd import *
import matplotlib
%matplotlib inline

In [ ]:
fname = '../M0r8_f2.h5'
prex = '../f2'

labels = [r'$\mathrm{t_{life}}$', r'$\log \delta$', r'$\lambda_0$', r'$\alpha$']

reader = emcee.backends.HDFBackend(fname)
ndim = len(labels)

tau = reader.get_autocorr_time(tol=1)
tau = np.max(tau); print('max tau:',tau)
Nburnin = int(3*tau)
Nthin = int(tau/2)

Nburnin = 0
Nthin = 1

In [ ]:
samples = reader.get_chain(discard=Nburnin)
probs = reader.get_log_prob(discard=Nburnin)
print('len of samples:', len(samples))

samples = reader.get_chain(discard=Nburnin, thin=Nthin, flat=True)
probs = reader.get_log_prob(discard=Nburnin, thin=Nthin, flat=True)
print('len of extracted samples:', len(samples))
theta_max = samples[np.argmax(probs)]
print('best paras:',theta_max,np.max(probs))


In [ ]:
ndraw = 20
fig, ax = plt.subplots(figsize=(10, 10)) #figsize=(10, 10)
curve_name = 'MF'
best_model = model(theta_max)
xs = best_model['M_BH'][::int(N_mf/100)] # ~100 points
y_data = best_model[curve_name+'_data'][::int(N_mf/100)]
y_logdata = np.log10(y_data)
y_best = best_model[curve_name][::int(N_mf/100)]
y_err = best_model[curve_name+'_data_err'][::int(N_mf/100)]
ax.plot(xs, y_data, c='C0',label='W10')
# error band of W10
plt.fill_between(xs,pow(10.,y_logdata-y_err/2.),pow(10.,y_logdata+y_err/2.),color='C0',alpha=0.5,label='_')

draw = np.floor(np.random.uniform(0,len(samples),size=ndraw)).astype(int)
thetas = samples[draw]
model_thetas = [model(theta_i) for theta_i in thetas]
mod_list = []
for i in range(ndraw):
    mod = model_thetas[i][curve_name][::int(N_mf/100)]
    mod_list.append(mod)
    # ax.plot(xs, mod, c='grey',label='_',alpha=.2)
spread = np.std(mod_list,axis=0)
med_model = np.median(mod_list,axis=0)
plt.fill_between(xs,med_model-spread,med_model+spread,color='orange',alpha=0.5,label=r'$1\sigma$ Posterior Spread')
ax.plot(xs, y_best, c='orange', label='Best-fit Model')
ax.set_xlim(1e7,1e10); ax.set_xscale('log')
ax.set_ylim(1e-10,1e-4); ax.set_yscale('log')
ax.legend(fontsize=fslegend)

plt.xlabel(r'$\mathrm{M_{BH}~(M_\odot)}$',fontsize=fslabel)
plt.ylabel(r'$\mathrm{\Phi_M~(Mpc^{-3}dex^{-1})}$',fontsize=fslabel)
plt.tick_params(labelsize=fstick)
plt.savefig(prex+'ndraw%dMF_spread.png'%ndraw,dpi=300,bbox_inches='tight')
# plt.show()

In [ ]:
ndraw = 40
fig, ax = plt.subplots(figsize=(10, 10)) #figsize=(10, 10)
curve_name = 'MF'
best_model = model(theta_max)
xs = best_model['M_BH'][::int(N_mf/100)] # ~100 points
y_data = best_model[curve_name+'_data'][::int(N_mf/100)]
y_logdata = np.log10(y_data)
y_best = best_model[curve_name][::int(N_mf/100)]
y_err = best_model[curve_name+'_data_err'][::int(N_mf/100)]
ax.plot(xs, y_data, c='C0',label='data')
# error band of W10
plt.fill_between(xs,pow(10.,y_logdata-y_err/2.),pow(10.,y_logdata+y_err/2.),color='C0',alpha=0.5,label='_')

draw = np.floor(np.random.uniform(0,len(samples),size=ndraw)).astype(int)
thetas = samples[draw]
model_thetas = [model(theta_i) for theta_i in thetas]
mod_list = []
for i in range(ndraw):
    mod = model_thetas[i][curve_name][::int(N_mf/100)]
    mod_list.append(mod)
    # ax.plot(xs, mod, c='grey',label='_',alpha=.2)
spread = np.std(mod_list,axis=0)
med_model = np.median(mod_list,axis=0)
plt.fill_between(xs,med_model-spread,med_model+spread,color='orange',alpha=0.5,label=r'$1\sigma$ Posterior Spread')
ax.plot(xs, y_best, c='orange', label='Best-fit Model')
ax.set_xlim(1e7,1e10); ax.set_xscale('log')
ax.set_ylim(1e-10,1e-4); ax.set_yscale('log')
ax.legend(fontsize=fslegend)

plt.xlabel(r'$\mathrm{M_{BH}~(M_\odot)}$',fontsize=fslabel)
plt.ylabel(r'$\mathrm{\Phi_M~(Mpc^{-3}dex^{-1})}$',fontsize=fslabel)
plt.tick_params(labelsize=fstick)
plt.savefig(prex+'ndraw%dMF_spread.png'%ndraw,dpi=300,bbox_inches='tight')
# plt.show()

In [ ]:
ndraw = 60
fig, ax = plt.subplots(figsize=(10, 10)) #figsize=(10, 10)
curve_name = 'MF'
best_model = model(theta_max)
xs = best_model['M_BH'][::int(N_mf/100)] # ~100 points
y_data = best_model[curve_name+'_data'][::int(N_mf/100)]
y_logdata = np.log10(y_data)
y_best = best_model[curve_name][::int(N_mf/100)]
y_err = best_model[curve_name+'_data_err'][::int(N_mf/100)]
ax.plot(xs, y_data, c='C0',label='data')
# error band of W10
plt.fill_between(xs,pow(10.,y_logdata-y_err/2.),pow(10.,y_logdata+y_err/2.),color='C0',alpha=0.5,label='_')

draw = np.floor(np.random.uniform(0,len(samples),size=ndraw)).astype(int)
thetas = samples[draw]
model_thetas = [model(theta_i) for theta_i in thetas]
mod_list = []
for i in range(ndraw):
    mod = model_thetas[i][curve_name][::int(N_mf/100)]
    mod_list.append(mod)
    # ax.plot(xs, mod, c='grey',label='_',alpha=.2)
spread = np.std(mod_list,axis=0)
med_model = np.median(mod_list,axis=0)
plt.fill_between(xs,med_model-spread,med_model+spread,color='orange',alpha=0.5,label=r'$1\sigma$ Posterior Spread')
ax.plot(xs, y_best, c='orange', label='Best-fit Model')
ax.set_xlim(1e7,1e10); ax.set_xscale('log')
ax.set_ylim(1e-10,1e-4); ax.set_yscale('log')
ax.legend(fontsize=fslegend)

plt.xlabel(r'$\mathrm{M_{BH}~(M_\odot)}$',fontsize=fslabel)
plt.ylabel(r'$\mathrm{\Phi_M~(Mpc^{-3}dex^{-1})}$',fontsize=fslabel)
plt.tick_params(labelsize=fstick)
plt.savefig(prex+'ndraw%dMF_spread.png'%ndraw,dpi=300,bbox_inches='tight')
# plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
curve_name = 'LF'
xs = best_model['M1450']
x_data = best_model['M1450_data']
y_data = best_model[curve_name+'_data']
y_data_err = best_model[curve_name+'_data_err']
y_best = best_model[curve_name]
ax.scatter(x_data, y_data, label='_')
plt.errorbar(x_data, y_data, yerr=y_data_err,fmt='o',capsize=10)
# print('y_data_err',y_data_err)
mod_list = []
for i in range(ndraw):
    mod = model_thetas[i][curve_name]
    mod_list.append(mod)
    # ax.plot(xs, mod, c='grey',label='_',alpha=.2)
spread = np.std(mod_list,axis=0)
med_model = np.median(mod_list,axis=0)
plt.fill_between(xs,med_model-spread,med_model+spread,color='orange',alpha=0.5,label='_')
ax.plot(xs, y_best, c='orange', label='_')
ax.text(-26,5, 'f2\n'+ \
labels[0]+' = %.2f Myr\n'%(theta_max[0])+labels[1]+' = %.2f\n'%(theta_max[1])\
+labels[2]+' = %.2f\n'%(theta_max[2])+labels[3]+' = %.2f\n'%(theta_max[3])
, fontsize=20)
ax.set_xlim(np.max(xs),np.min(xs))
ax.set_ylim(5e-3,1e2)
ax.set_yscale('log')
plt.xlabel(r'$\mathrm{M_{1450}}$',fontsize=fslabel)
plt.ylabel(r'$\mathrm{\Phi~(Gpc^{-3}mag^{-1})}$',fontsize=fslabel)
plt.tick_params(labelsize=fstick)
# plt.tight_layout()
plt.savefig(prex+'ndraw%dLF_spread.png'%ndraw,dpi=300,bbox_inches='tight')
